#Agentic RAG
Agentic Retrieval-Augmented Generation (RAG) is an advanced framework designed to handle complex information retrieval tasks using a network of intelligent agents. These agents collaborate to perform nuanced tasks such as synthesizing information from multiple documents, summarizing content, and comparing data points across various sources. Agentic RAG infuses autonomy and intelligence into traditional retrieval systems, enabling them to act as passive tools and proactive entities that understand context, evaluate data quality, and make informed decisions.

## Blog

For a detailed explanation of agentic rag, check out  [blog post on Medium](https://aksdesai1998.medium.com/662bac582da9).


In [ ]:
# install the required dependencies
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub lancedb  langchain langgraph langchain-text-splitters langchain_openai gradio

In [ ]:
import os
import getpass
import gradio as gr
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import LanceDB
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph.message import add_messages
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolExecutor, ToolNode, tools_condition

In [ ]:
# Function to set environment variables securely
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

# (Optional) For tracing
os.environ["LANGCHAIN_TRACING_V2"] = "False"
_set_env("LANGCHAIN_API_KEY")


# upload the data based on your usecase

urls = [
    "https://content.dgft.gov.in/Website/CIEP.pdf",
    "https://content.dgft.gov.in/Website/GAE.pdf",
    "https://content.dgft.gov.in/Website/HTE.pdf",
]


docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to  lancedb as vectordb

vectorstore = LanceDB.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()


# create the tools
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about customs import export procedure,GST & EXPORTS , How to export",
)

tools = [retriever_tool]
tool_executor = ToolExecutor(tools)


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


def grade_documents(state) -> Literal["generate", "rewrite"]:
    class grade(BaseModel):
        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
    llm_with_tool = model.with_structured_output(grade)
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]
    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"question": question, "context": docs})
    score = scored_result.binary_score

    return "generate" if score == "yes" else "rewrite"


def agent(state):
    messages = state["messages"]
    model = ChatOpenAI(temperature=0, streaming=True, model="gpt-4-turbo")
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}


def rewrite(state):
    messages = state["messages"]
    question = messages[0].content
    msg = [
        HumanMessage(
            content=f""" \n
            Look at the input and try to reason about the underlying semantic intent / meaning. \n
            Here is the initial question:
            \n ------- \n
            {question}
            \n ------- \n
            Formulate an improved question: """,
        )
    ]
    model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
    response = model.invoke(msg)
    return {"messages": [response]}


def generate(state):
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]
    docs = last_message.content

    prompt = hub.pull("rlm/rag-prompt")
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = prompt | llm | StrOutputParser()
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


workflow = StateGraph(AgentState)
workflow.add_node("agent", agent)
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)
workflow.add_node("rewrite", rewrite)
workflow.add_node("generate", generate)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent", tools_condition, {"tools": "retrieve", END: END}
)
workflow.add_conditional_edges("retrieve", grade_documents)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")
graph = workflow.compile()


def process_message(user_message):
    inputs = {"messages": [("user", user_message)]}
    content_output = None
    for output in graph.stream(inputs):
        print(f"Debug output: {output}")  # Debugging line to print the output
        if "agent" in output and "messages" in output["agent"]:
            messages = output["agent"]["messages"]
            if messages and hasattr(messages[0], "content"):
                content_output = messages[0].content  # Accessing attribute directly
                print(f"Extracted content: {content_output}")  # Print extracted content
    return content_output if content_output else "No relevant output found."


# Define example questions to guide the user
example_questions = [
    "explain me in short what is PM Gati Shakti National Master Plan (NMP)?"
]

# Create a Gradio interface
iface = gr.Interface(
    fn=process_message,
    inputs="text",
    outputs="text",
    title="Agentic RAG ",
    description="Enter a message to query related to export import .",
    examples=example_questions,
)

# Launch the Gradio app
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1300bf30884d92867e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Debug output: {'agent': {'messages': [AIMessage(content='The PM Gati Shakti National Master Plan (NMP) is an ambitious initiative launched by the Government of India aimed at improving infrastructure development across the country. Launched by Prime Minister Narendra Modi, the plan seeks to integrate the planning and coordination of various infrastructure projects across different sectors and ministries.\n\nThe core objective of the Gati Shakti NMP is to enhance multi-modal connectivity and reduce logistics costs by bringing together rail, road, air, and waterways projects under a single, unified framework. This holistic approach is intended to boost economic growth, create jobs, and promote regional connectivity.\n\nThe plan utilizes digital technology to map and synchronize projects, ensuring that all related departments and stakeholders are aligned, which helps in eliminating bottlenecks, improving project execution speed, and enhancing overall efficiency. The Gati Shakti NMP is see

#some quetions for testing
explain me in short what is PM Gati Shakti National Master Plan (NMP)?

what is Zero Rating of Exports?

what is Export Inspection Council of India?

please give us some Details of some of the major initiatives /schemes please ?